In [1]:
%pip install pandas numpy pyarrow tqdm lxml razdel

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from razdel import tokenize
import pandas as pd
import numpy as np

In [3]:
SEED = 42

np.random.seed(SEED)

In [4]:
with open("dataset_1937770_3.txt", "r", encoding="utf8") as in_file:
    with open("dataset.txt", "w", encoding="utf8") as out_file:
        for line in in_file.readlines():
            out_file.write(line.replace(",", ";", 1))

In [5]:
data = pd.read_csv("dataset.txt", sep=";", engine="python", on_bad_lines="skip")

random_idxs = np.random.randint(0, 1000, 10)

data["text_no_spaces"][np.isin(data["id"], random_idxs)]

20                 ищуподработкуповечерам
71                   ищуработуофицианткой
102          куплюковрикдляйоги,недорого!
106    ищудрузейдляпутешествий,летомвгоры
121         ищустоликдляноутбука,складной
270                     куплюгитаруGibson
435               Когданачасахважныминуты
614                       Верхомназвезде,
700                     Ждутегодомавсегда
860      Янивчемнеоткажу,янивчёмнеоткажу!
Name: text_no_spaces, dtype: object

Во время экспериментов пробовался более продвинутый токенайзер, который разделяет по смысловым единицам, а не пробелам и символам, но на сыром тексте без пробелов он показывает себя не очень хорошо. В связи с этим, используется более простой токенайзер, который, по сути, полезен только чтоб отделять символы.

In [10]:
corpus = data["text_no_spaces"]

for text in corpus[random_idxs]:
	tokens = [token.text for token in tokenize(text)]
	print(tokens)

['куплюковрикдляйоги', ',', 'недорого', '!']
['Когданачасахважныминуты']
['Янивчемнеоткажу', ',', 'янивчёмнеоткажу', '!']
['куплюгитару', 'Gibson']
['ищудрузейдляпутешествий', ',', 'летомвгоры']
['ищуработуофицианткой']
['Ждутегодомавсегда']
['ищуподработкуповечерам']
['Верхомназвезде', ',']
['ищустоликдляноутбука', ',', 'складной']


Токенайзером на глаз видно, что тексты делятся очень плохо. Кажется, не имеет большого смысла использовать его в качестве предобработки сырых данных, а лучше использовать другой подход. (причем, ладно бы он делал только лишние разбиения. Можно было бы тогда свести задачу к попытке склеить лишние разбиения. Но с такой работой, которая наблюдается, он и лишних разбиений много сделал, и не везде там, где надо, разбил)

У нас есть задача: поделить строки таким образом, чтобы разделенный текст был осмысленным, а значит наиболее вероятным с точки зрения встречаемости слов вместе.

то есть, пусть есть строка "куплюковрик". Можно поделить ее огромным количеством способов пробелами. При этом разбиение "куплю коврик" должно быть более приоритетным, чем какое бы то ни было другое, например "куп люков рик". Приоритетность отражается большей вероятностью встречи слов в датасете с запросами пользователей. Пусть разбиение текста запроса без пробелов -- это слова через пробел $x = \{x_1, .., x_n\}$. Тогда истинное разбиение $x^* = \{x_1^*, .., x_k^*\}$ должно быть более вероятное (скорее всего не всегда и можно привести примеры, но приблизительно должно выполняться):

$$
	p(x^*) \gtrsim p(x) \quad \forall x \neq x^*,
$$

где
$$
\begin{split}
	p(x^*) \equiv p(x_1^*, ... , x_k^*) = p(x_1^*) \cdot p(x_2^* | x_1^*) \cdot ... \cdot p(x_k^* | x_1^*, ... x_{k-1}^*), \\
	p(x) \equiv p(x_1, ... , x_n) = .... \text{аналогично}
\end{split}
$$

Таким образом, зная все условные вероятности встретить некоторое слово $y_1$ в контексте других слов $\{y_i\}_{i=2}^{m}$ в текстах пользовательских запросов, можно разрешить эту задачу.

Естественно, знать все такие вероятности невозможно, поэтому нужно прибегнуть к некоторым упрощениям.

Первое допущение типа марковского. Считаем, что для больших последовательностей слова из начала не сильно влияют на вероятность слов с конца, то есть, например, контекста $l$ слов перед исходным с хорошей точностью хватит для оценки, поскольку $p(x_i | x_{i-1}, x_{i-2}, ... , x_{1}) \approx p(x_i | \underbrace{x_{i-1}, x_{i-2}, ... , x_{i-l}, x_{i-l - 1}}_{l})$

На самом деле даже подход с $l=0$ (униграммы) отсеит много мусора, так как неправильное разбиение, в котором участвует слово "рик" из-за одной только вероятности $p("рик") \approx 0$ сведет к малому значению или нулю вероятность всего разбиения $p("куп", "люков", "рик") \approx p("куп") \cdot p("люков") \cdot p("рик") \approx 0$. Вероятность же просто адекватного разбиения на осмысленные слова будет больше этого.

Однако, было бы здорово из двух адекватных разбиений (которые, вероятно, могут все таки встречаться) выбирать более подходящее по смыслу именно к запросам пользователей.

Самое сложное в этом всем -- найти открытый датасет, который по статистике был бы похож на пользовательские запросы, названия и описания со словами, написанными транслитом ("айфон" или "оппо") и подобными артефактами.

Вроде то, что надо: https://www.kaggle.com/datasets/antonoof/avito-data. Нет запросов правда, только названия и описания объявлений, но что-то лучше вряд ли получиться найти. Также скачаем корпус русскоязычных текстов: https://www.opencorpora.org/.

In [8]:
# %pip install kagglehub[pandas-datasets]

In [9]:
# import kagglehub
# from kagglehub import KaggleDatasetAdapter
# from tqdm.notebook import tqdm

In [10]:
# file_path = "avito_data.csv"

# df = kagglehub.dataset_load(
# 	KaggleDatasetAdapter.PANDAS,
# 	"antonoof/avito-data",
# 	file_path,
# )


*НЕ РАБОТАЕТ, ВЫДАЕТ SSLEOFError*

*ПРИ ПРОВЕРОЧНОМ ЗАПУСКЕ СКАЧАЙТЕ ДАТАСЕТЫ ВРУЧНУЮ И ПОМЕСТИТЕ АРХИВЫ РЯДОМ С НОУТБУКОМ*

In [11]:
from pathlib import Path
import zipfile
from tqdm import tqdm

In [12]:
zip_path = Path('archive.zip')
extract_path = Path('avito_data')
extract_path.mkdir(exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

Нужно сильно постараться, чтоб статистика по словам в таком датасете была похожа на статистику в словах реальных пользователей, ведь тут намеренно приведено много примеров очень похожих, использующих одинаковые слова. Однако, фильтровать все тексты, похожие по индексу Жаккара, например, будет очень долго. Поэтому, в трейновой выборке удалим просто примеры, у которых дублируется id и примеры, где стоит флаг is_double, а в тестовой выборке удалим просто всех кандидатов и объявления с одинаковым id.

In [13]:
train_files = list((extract_path / "data").glob("train_part_*.snappy.parquet"))
train_cols = ['base_item_id', 'cand_item_id', 'is_double', 'base_title', 'cand_title', 'base_description', 'cand_description']
train_df = pd.concat([pd.read_parquet(f, columns=train_cols) for f in tqdm(train_files)], ignore_index=True)

doubles_df = train_df[train_df['is_double'] == 1]
blacklisted_ids = set(doubles_df['base_item_id']).union(set(doubles_df['cand_item_id']))

base_train = train_df[['base_item_id', 'base_title', 'base_description']].rename(columns={'base_item_id': 'item_id', 'base_title': 'title', 'base_description': 'description'})
cand_train = train_df[['cand_item_id', 'cand_title', 'cand_description']].rename(columns={'cand_item_id': 'item_id', 'cand_title': 'title', 'cand_description': 'description'})
all_train_df = pd.concat([base_train, cand_train], ignore_index=True).drop_duplicates()

cleaned_train_df = all_train_df[~all_train_df["item_id"].isin(blacklisted_ids)]

100%|██████████| 4/4 [00:16<00:00,  4.02s/it]


In [14]:
test_files = list((extract_path / "data").glob("test_part_*.snappy.parquet"))
test_cols = ['base_item_id', 'base_title', 'base_description']
test_df = pd.concat([pd.read_parquet(f, columns=test_cols) for f in tqdm(test_files)], ignore_index=True)

base_test = test_df.rename(columns={'base_item_id': 'item_id', 'base_title': 'title', 'base_description': 'description'})
cleaned_test_df = base_test.drop_duplicates(subset=['item_id'])

100%|██████████| 2/2 [00:03<00:00,  1.76s/it]


In [15]:
full_df = pd.concat([cleaned_train_df, cleaned_test_df], ignore_index=True).drop_duplicates(subset=['item_id']).reset_index(drop=True)

In [16]:
full_df[["title", "description"]][np.isin(full_df.index, random_idxs)]

,title,description
20,Подголовники priora,В хоpошем cоcтоянии \nЦенa зa штуку\nПapa 900
71,Коcтюм для тaнцев,"Пpодaю коcтюм для тaнцев фееpия, зaкaзaли нa п..."
102,Хоккейные тpуcы Easton Synergy 900 Sr L,🏒Хоккейные тpуcы Easton Synergy 900\n\n🔸Рaзмеp...
106,Стульчик для коpмления,"cтульчик в отличном cоcтоянии. 3 позы, можно п..."
121,Бюcтгaльтеp Intimissimi новый,Новый бюcтгaльтеp Intimissimi модель giada 80Б...
270,Лыжные ботинки nordway 40,Пpодaю лыжные ботинки в идеaльном cоcтоянии.Рa...
435,Пpоклaдки для гpуди,В оcтaтке 19 шт.Кaждaя в индивидуaльной упaков...
614,Сaлфеткa для pук и лицa мaхpa вт боpдовый по ц,Вид товapa: Сaлфеткa\nНaзнaчение : для pук и ...
700,Зaбоp cеткa 3D,Зaбоp cеткa 3d\n\n👍 3д cеткa в нaличии и под з...
860,Лего Мaйнкpaфт Деpевня 778 детaлей,Лего Мaйнкpaфт Деpевня 778 детaлей


Теперь надо сделать предобработку, удалить хотя бы указанные дубликаты и привести все слова к нижнему регистру.

In [17]:
full_df.shape[0]

2516491

Перейдем ко второму датасету

In [18]:
import xml.etree.ElementTree as ET

In [19]:
zip_path = Path("annot.opcorpora.xml.zip")
extract_path = Path("open_corpora_data")
extract_path.mkdir(exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [21]:
xml_corpus_file = extract_path / "annot.opcorpora.xml"
tree = ET.parse(xml_corpus_file)
root = tree.getroot()

xml_texts = []
for text_element in tqdm(root.findall('.//text')):
    name_text = text_element.get('name', '')
    source_texts = [source.text for source in text_element.findall('.//source') if source.text]

    full_text_parts = [name_text] + source_texts
    full_text = " ".join(part.strip() for part in full_text_parts if part and part.strip())
    
    if full_text:
        xml_texts.append(full_text)

data_for_df = {
    'item_id': [f'xml_corpus_{i}' for i in range(len(xml_texts))],
    'title': '',
    'description': xml_texts
}
additional_df = pd.DataFrame(data_for_df)
additional_df

100%|██████████| 4007/4007 [00:00<00:00, 7164.73it/s]


,item_id,title,description
0,xml_corpus_0,,"""Частный корреспондент"""
1,xml_corpus_1,,00021 Школа злословия «Школа злословия» учит п...
2,xml_corpus_2,,00022 Последнее восстание в Сеуле «Последнее в...
3,xml_corpus_3,,00023 За кота - ответишь! За кота – ответишь! ...
4,xml_corpus_4,,00024 Быстротечный кинороман Быстротечный кино...
...,...,...,...
4002,xml_corpus_4002,,Мягкое нёбо Мягкое нёбо Мягкое нёбо или нёбная...
4003,xml_corpus_4003,,"Небо Небо Небо (производные небеса, небосвод) ..."
4004,xml_corpus_4004,,Звёздное небо Звёздное небо Звёздное небо — со...
4005,xml_corpus_4005,,Диффузное излучение неба Диффузное излучение н...


In [22]:
full_df = pd.concat([full_df, additional_df], ignore_index=True)

Вроде нормально, теперь сделаем из этого единый корпус текста

In [23]:
full_df["text"] = full_df["title"] + ". " + full_df["description"]
full_df.drop(["title", "description"], axis=1)
full_df

,item_id,title,description,text
0,5aa8b125a3218ee10d56452d6e3fc6b05b92acae7e0710...,Коcтюм женcкий,Новый комплект,Коcтюм женcкий. Новый комплект
1,9f9b68e75034a161c116e764605db3275b514213c04ec7...,Школьный коcтюм для мaльчикa 134,"Школьный коcтюм нa мaльчикa чеpного цветa, paз...",Школьный коcтюм для мaльчикa 134. Школьный коc...
2,538538e5282be1b3aa4cad2feb0000cadc9d4d6602b955...,Иcлaмcкaя кapтинa из эпокcидной cмолы,Я pиcую иcлaмcкиe кapтины из эпокcидной cмoлы ...,Иcлaмcкaя кapтинa из эпокcидной cмолы. Я pиcую...
3,30f2136fc45172af7f66d4993132c05649c563c720dc52...,Оpигинaл Nike Air Jordan 1 High OG University ...,Мы зaключили договоp c официaльным диcтpибьюто...,Оpигинaл Nike Air Jordan 1 High OG University ...
4,4afe50981f5dded76ad9c4357a1df34dcf28f6248e2ccf...,Кpоccовки Nike Air Jordan 1 Low OG Travis Scot...,Nike Air Jordan 1 Low OG Travis Scott Black Ph...,Кpоccовки Nike Air Jordan 1 Low OG Travis Scot...
...,...,...,...,...
2520493,xml_corpus_4002,,Мягкое нёбо Мягкое нёбо Мягкое нёбо или нёбная...,. Мягкое нёбо Мягкое нёбо Мягкое нёбо или нёбн...
2520494,xml_corpus_4003,,"Небо Небо Небо (производные небеса, небосвод) ...",". Небо Небо Небо (производные небеса, небосвод..."
2520495,xml_corpus_4004,,Звёздное небо Звёздное небо Звёздное небо — со...,. Звёздное небо Звёздное небо Звёздное небо — ...
2520496,xml_corpus_4005,,Диффузное излучение неба Диффузное излучение н...,. Диффузное излучение неба Диффузное излучение...


In [24]:
import re

In [28]:
text = full_df["text"].tolist()

corpus = []

def preprocess(line):
	line = line.lower().replace("ё", "е")
	line = re.sub(r"\s+", " ", line).strip()
	return line


for line in tqdm(text):
	cleaned_line = preprocess(line)
	if len(cleaned_line):
		corpus.append(cleaned_line)

with open("clean_corpus.txt", "w", encoding="utf8") as file:
	for line in tqdm(corpus):
		file.write(line + "\n")

100%|██████████| 2520498/2520498 [01:05<00:00, 38725.92it/s]


Поскольку датасет был для различия дубликатов объявлений, некоторые названия и описания повторяются чаще, чем следует, но сильно это не должно помешать. (но вообще мусора многовато осталось, на итоговом качестве это может сказаться)

Перейдем непосредственно к модели. Первоначально, для использования статистики по униграммам и биграммам в корпусе, нужно их посчитать, а потом для определения вероятностей использовать формулы:

$$
	p(\text{word}_2 | \text{word}_1) = \frac{p(\text{word}_1, \text{word}_2)}{p(\text{word}_1)}
$$

где

$$
p(\text{word}_i) = \frac{n_{\text{word}_i}}{n}, \quad p(\text{word}_i, \text{word}_{i-1}) = \frac{n_{\text{word}_i, \text{word}_{i-1}}}{n}
$$

тогда юзабельная формула:

$$
p(\text{word}_i | \text{word}_{i-1}) = \frac{n_{\text{word}_i, \text{word}_{i-1}}}{n_{\text{word}_{i-1}}}
$$

Перейдем к подсчетам

In [29]:
from collections import defaultdict

In [34]:
unigram_counts = defaultdict(int)
bigram_counts = defaultdict(int)
total_words = 0

with open("clean_corpus.txt", "r", encoding='utf8') as file:
    for line in tqdm(file, total=2_500_000):
        tokens = [token.text for token in tokenize(line)]
        
        if not tokens:
            continue
            
        total_words += len(tokens)
        
        for word in tokens:
            unigram_counts[word] += 1

        for i in range(len(tokens) - 1):
            bigram = (tokens[i], tokens[i+1])
            bigram_counts[bigram] += 1

print(f"количество слов: {total_words}")
print(f"уникальных слов: {len(unigram_counts)}")
print(f"уникальных биграмм: {len(bigram_counts)}")

2520498it [49:43, 844.80it/s]                               

количество слов: 867914283
уникальных слов: 2205919
уникальных биграмм: 14557985


*Как-то очень медленно с этим токенизатором все работает. Такое ощущение, что вручную написать код для разделения на токены было бы быстрее*

In [759]:
print(unigram_counts["виво"], unigram_counts["сяоми"], unigram_counts["ксяоми"], unigram_counts["ксиоми"])

32 189 0 2


In [39]:
print(bigram_counts[("смартфон", "vivo")], bigram_counts[("телефон", "vivo")])
print(bigram_counts[("хорошем", "состоянии")], bigram_counts[("плохом", "состоянии")])

0 13
152 1


Видно, что датасет довольно маленький, чтобы с хорошей точностью восстановить статистику. Слова, которые точно должны встречаться среди запросов вместе, например "смартфон vivo", в датасет не входят, так что $p(\text{"смартфон"} | \text{"vivo"})$ будет строгим нулем вместо некоторого малого значения.

Теперь нужно придумать, как пользоваться полученными значениями эффективно. Для строки длины $l$ количество разбиений зависит от $l$ экспоненциально, так что просто перебор всех может оказаться слишком долгим. Нужно придумать нечто более эффективное, используя подход динамического программирования.

Решение:

* Пусть мы рассматриваем префикс из $i$ первых букв. Тогда для определения лучшего его разбиения будем рассматривать все его суффиксы (буквы от некоторого $j$ до $i$) и считать скор разбиения с последним словом $[j:i]$. Для этого нужно знать скор лучшего разбиения префикса $[:j]$ и вероятность нового слова $[j:i]$ при условии последнего слова из лучшего разбиения префикса $[:j]$. Далее нужно будет выбрать лучшее разбиение по скору и сохранить его.

* При этом, скор рассчитыватся, как
$$
score(x) := \log p(x) = \log p(x_1) + \sum \log p(x_i | x_{i-1})
$$

* Далее переходим к следующему символу и так же перебираем разбиения префикса.

* Таким образом, у нас получается таблица со скорами лучшего разбиения всех префиксов строки и с помощью нее мы сможем восстановить это разбиение.

In [40]:
model_data = {
    "unigram_counts": dict(unigram_counts),
    "bigram_counts": dict(bigram_counts),
    "total_words": total_words,
}

In [762]:
def len_bonus(word_len):
    return (word_len - 1) ** 2 / 2

In [763]:
class Splitter:
	def __init__(self, model_data=model_data):
		self.unigrams = model_data["unigram_counts"]
		self.bigrams = model_data["bigram_counts"]
		self.total = model_data["total_words"]
		self.log_total = np.log10(self.total)
		self.vocabular = set(self.unigrams.keys())

	def unigram_score(self, word):
		count = self.unigrams.get(word, 0)
		if count == 0:
			return -np.inf
		return np.log10(count) - self.log_total

	def bigram_score(self, word_1, word_2):
		bigram_count = self.bigrams.get((word_1, word_2), 0)
		word1_count = self.unigrams.get(word_1, 0)

		#  если биграммы нет, считаем слова независимыми, тогда скор пары - это просто сумма скоров
		if bigram_count > 0 and word1_count > 0:
			return np.log10(bigram_count) - np.log10(word1_count)
		else:
			return self.unigram_score(word_1) + self.unigram_score(word_2)

	def split(self, text):
		mem = {0: (0., 0)}

		for i in range(1, len(text) + 1):
			best_score = -np.inf
			best_split_pos = 0

			for j in range(max(0, i-20), i):
				word = text[j:i]

				if word in self.vocabular:
					prev_score, prev_best_split_pos = mem.get(j, (-np.inf, 0))
					if prev_score == -np.inf:
						continue

					prev_word = text[mem[j][1]:j] if j > 0 else None

					log_prob = self.unigram_score(word) if prev_word is None else self.bigram_score(prev_word, word)

					new_score = prev_score + log_prob + len_bonus(len(word))

					if new_score > best_score:
						best_score = new_score
						best_split_pos = j

			mem[i] = (best_score, best_split_pos)

		if mem[len(text)][0] == -np.inf:
			return text
		else:
			result = []
			i = len(text)
			while i != 0:
				j = mem[i][1]
				result.append(text[j:i])
				i = j

			result.reverse()
			return " ".join(result)


In [764]:
splitter = Splitter(model_data)

In [765]:
print(splitter.split("айфон16промах"))
print(splitter.split("айфон16промакс"))
print(splitter.split("айфон18промакс"))
print(splitter.split("продамколесонедорого"))

айфон 16 промах
айфон 16 про макс
айфон 1 8 про макс
продам колесо недорого


Выглядит неплохо. Весов немало -- модель на 700МБ, зато запускается без гпу без проблем.

In [766]:
# import pickle

# model_path = Path('bigram_model.pkl')
# with model_path.open('wb') as f:
#     pickle.dump(model_data, f)

Теперь практически все готово для разбиения предоставленного датасета.

In [767]:
def preprocess_and_map(text):
    clean_text_parts = []
    clean_to_original_map = []

    for token in tokenize(text):
        normalized_token = token.text.lower().replace('ё', 'е')
        
        clean_text_parts.append(normalized_token)
        
        for _ in range(len(normalized_token)):
            clean_to_original_map.append(token.start)

    clean_text_for_splitter = "".join(clean_text_parts)

    return clean_text_for_splitter, clean_to_original_map

In [768]:
def post_process_punctuation(text):
	text = re.sub(r'\s([?.!,:;")])', r'\1', text)
	
	text = re.sub(r'([("])\s', r'\1', text)

	text = re.sub(r'(\S)-\s', r'\1 - ', text)
	text = re.sub(r'\s-(\S)', r' - \1', text)
	
	return text


In [777]:
task_data = pd.read_csv("dataset.txt", sep=";", engine="python")

all_predicted_positions = []
all_predicted_texts = []

for text_no_spaces in tqdm(task_data['text_no_spaces']):
    clean_text, mapping = preprocess_and_map(text_no_spaces)
    
    if not clean_text:
        all_predicted_positions.append("")
        all_predicted_texts.append(text_no_spaces)
        continue

    raw_split_text = splitter.split(clean_text)

    final_split_text = post_process_punctuation(raw_split_text)
    all_predicted_texts.append(final_split_text)

    tokens = raw_split_text.split(' ')
    positions = []
    current_clean_pos = 0
    
    for token in tokens[:-1]:
        current_clean_pos += len(token)
        
        if current_clean_pos < len(mapping):
            original_space_position = mapping[current_clean_pos]
            positions.append(str(original_space_position))

    positions_str = " ".join(positions)
    all_predicted_positions.append(positions_str)


submission_df = task_data.copy()
submission_df["predicted_text"] = all_predicted_texts
submission_df["predicted_positions"] = all_predicted_positions
submission_df.to_csv("submission.csv", index=False)


print(submission_df[["text_no_spaces", "predicted_text"]][submission_df["id"].isin(np.random.randint(0, 1000, 10))])

100%|██████████| 1005/1005 [00:00<00:00, 3055.40it/s]

                                  text_no_spaces  \
87                             сдаюквартирусемье   
99              продамшкаф,почтиновый,сзеркалами   
130                  ищустолярныеуслуги,недорого   
214                      новаяплитаэлектрическая   
330                       ищуучителяфранцузского   
372                              SamsungA54новый   
458                  Городскомландшафтеменязовёт   
466                          Телоисполняетсяогня   
663              Когдалюдинаулицахивокнахквартир   
871  'Друзьяхотятпокушать,пойдём,приятель,влес!'   

                                        predicted_text  
87                                сдаю квартиру с емье  
99                продам шкаф, почтиновый, с зеркалами  
130                     ищу столярные услуги, недорого  
214                         новая плит а электрическая  
330                           ищу учителя французского  
372                                  samsunga 54 новый  
458                     горо

*Как для модели на биграммах, которая работает без гпу и чего-то сложного, получилось неплохо, но от идеала очень далеко. Сказываются и артефакты датасета, на котором обучалась модель. Довольно часто можно встретить неправильные слова, при этом некоторых специфических слов, редких, зачастую не находится.*

*Для улучшения, можно попробовать результаты уже этой модели подавать на вход некоторой нейросетевой, однако*